# 使用FP-growth算法来高效发现频繁项集（FP：频繁模式）

1. 基于Apriori构建
2. 执行速度快于Apriori，性能要好两个数量级以上
3. 能够高效地发现频繁项集，但是不能用于发现关联规则

## 12.1 FP树：用于编码数据集的有效方式
1. 优点： 一般要快于Apriori
2. 缺点： 实现比较困难，在某些数据集上性能会下降
3. 适用数据类型：标称型数据

## FP-gowth的一般流程
1. 收集数据：使用任意方法
2. 准备数据：由于存储的是集合，所以需要离散数据，如果要处理连续数据，需要将它们量化为离散值
3. 分析数据：使用任意方法
4. 训练算法：构建一个FP树，并对树进行挖掘
5. 测试算法：没有测试过程
6. 使用算法：可用于识别经常出现的元素项，从而用于指定决策，推荐元素或进行预测等应用中

## 12.2 构建FP树

In [1]:
## FP树的类定义
class treeNode:
    def __init__(self,nameValue,numOccur,parentNode):
        # 存放节点的名字变量
        self.name = nameValue
        self.count = numOccur
        # 用于链接相似的元素项
        self.nodeLink = None
        self.parent = parentNode
        self.children = {}
        
    def inc(self,numOccur):
        # 给count变量增加给定值
        self.count += numOccur
        
    def disp(self,ind=1):
        print(' '*ind,self.name,'',self.count)
        for child in self.children.values():
            child.disp(ind+1)

In [2]:
rootNode = treeNode('pyramid',9,None)
rootNode.children['eye'] = treeNode('eye',13,None)
rootNode.disp()

  pyramid  9
   eye  13


In [3]:
rootNode.children

{'eye': <__main__.treeNode at 0x10908f6a0>}

In [4]:
rootNode.children['phoenix'] = treeNode('phoenix',3,None)
rootNode.disp()

  pyramid  9
   eye  13
   phoenix  3


## 构建FP树

In [5]:
## FP树构建函数
def createTree(dataSet,minSup=1):
    headerTable = {}
    #遍历一遍扫描数据集并统计每个元素项出现的频度，将这些信息保存在头指针中
    for trans in dataSet:
        for item in trans:
            # get()返回指定键的值，如果不存在，则返回默认值
            headerTable[item] = headerTable.get(item,0) + dataSet[trans]
    # 接着扫描头指针表删除那些出现次数小于minSup的项。
    for k in list(headerTable.keys()):
        if headerTable[k] < minSup:
            del(headerTable[k])
    freqItemSet = set(headerTable.keys())
    # 如果所有项都不频繁，无需下一步处理
    if len(freqItemSet) == 0:
        return None,None
    # 对头指针表稍加扩展以便可以保存计数值及指向每种类型第一个元素项的指针
    for k in headerTable:
        headerTable[k] = [headerTable[k],None]
    # 创建只包含空集合的根节点
    retTree = treeNode('Null Set',1,None)
    for tranSet,count in dataSet.items():
        localD = {}
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        if len(localD) > 0:
#             print(localD.items())
            orderedItems = [v[0] for v in sorted(localD.items(),key=lambda p:p[1],reverse=True)]
#             print(orderedItems)
            updateTree(orderedItems,retTree,headerTable,count)
#     print(headerTable)
    return retTree,headerTable

def updateTree(items,inTree,headerTable,count):
    if items[0] in inTree.children:
        inTree.children[items[0]].inc(count)
    else:
        inTree.children[items[0]] = treeNode(items[0],count,inTree)
        if headerTable[items[0]][1] == None:
            headerTable[items[0]][1] = inTree.children[items[0]]
        else:
            updateHeader(headerTable[items[0]][1],inTree.children[items[0]])
    if len(items) > 1:
        updateTree(items[1::],inTree.children[items[0]],headerTable,count)
        
def updateHeader(nodeToTest,targetNode):
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    nodeToTest.nodeLink = targetNode
            

In [6]:
# 导入数据集
def loadSimDat():
    simpDat = [['r','z','h','j','p'],
              ['z','y','x','w','v','u','t','s'],
              ['z'],
              ['r','x','n','o','s'],
              ['y','r','x','z','q','t','p'],
              ['y','z','x','e','q','s','t','m']]
    return simpDat

def createInitSet(dataSet):
    retDict = {}
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

In [7]:
simpDat = loadSimDat()
simpDat

[['r', 'z', 'h', 'j', 'p'],
 ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
 ['z'],
 ['r', 'x', 'n', 'o', 's'],
 ['y', 'r', 'x', 'z', 'q', 't', 'p'],
 ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]

In [8]:
initSet = createInitSet(simpDat)
initSet

{frozenset({'z'}): 1,
 frozenset({'h', 'j', 'p', 'r', 'z'}): 1,
 frozenset({'s', 't', 'u', 'v', 'w', 'x', 'y', 'z'}): 1,
 frozenset({'n', 'o', 'r', 's', 'x'}): 1,
 frozenset({'p', 'q', 'r', 't', 'x', 'y', 'z'}): 1,
 frozenset({'e', 'm', 'q', 's', 't', 'x', 'y', 'z'}): 1}

In [9]:
myFPtree,myHeaderTab = createTree(initSet,3)
myFPtree.disp()
myHeaderTab['t'][1].nodeLink.parent.name

  Null Set  1
   z  5
    r  1
    x  3
     t  3
      y  3
       s  2
       r  1
   x  1
    r  1
     s  1


AttributeError: 'NoneType' object has no attribute 'parent'

In [13]:
## 发现以给定元素项结尾的所有路径的函数
def ascendTree(leafNode,prefixPath):
    if leafNode.parent != None:
        prefixPath.append(leafNode.name)
        ascendTree(leafNode.parent,prefixPath)
        
def findPrefixPath(basePat,treeNode):
    # 条件模式基字典
    condPats = {}
    while treeNode != None:
        prefixPath = []
        ascendTree(treeNode,prefixPath)
        if len(prefixPath) > 1:
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        treeNode = treeNode.nodeLink
    return condPats

In [21]:
## 创建条件FP树
## 递归寻找频繁项集的mineTree函数
def mineTree(inTree,headerTable,minSup,preFix,freqItemList):
    # 对头指针表中的元素按照其出现频率进行排序，按从小到大
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p:p[1][0])]
    print("bigL:",bigL)
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        freqItemList.append(newFreqSet)
        condPattBases = findPrefixPath(basePat,headerTable[basePat][1])
        myCondTree,myHead = createTree(condPattBases,minSup)
        print('myHead:',myHead)
        print('newFreqSet:',newFreqSet)
        if myHead != None:
            mineTree(myCondTree,myHead,minSup,newFreqSet,freqItemList)
            print('conditional tree for:',newFreqSet)
            myCondTree.disp(1)

In [22]:
freqItems = []
mineTree(myFPtree,myHeaderTab,3,set([]),freqItems)

bigL: ['r', 't', 'y', 's', 'x', 'z']
myHead: None
newFreqSet: {'r'}
myHead: {'z': [3, <__main__.treeNode object at 0x10912a2e8>], 'x': [3, <__main__.treeNode object at 0x10912a320>]}
newFreqSet: {'t'}
bigL: ['z', 'x']
myHead: None
newFreqSet: {'z', 't'}
myHead: {'z': [3, <__main__.treeNode object at 0x109129278>]}
newFreqSet: {'x', 't'}
bigL: ['z']
myHead: None
newFreqSet: {'z', 'x', 't'}
conditional tree for: {'x', 't'}
  Null Set  1
   z  3
conditional tree for: {'t'}
  Null Set  1
   z  3
    x  3
myHead: {'z': [3, <__main__.treeNode object at 0x109129400>], 'x': [3, <__main__.treeNode object at 0x109129240>], 't': [3, <__main__.treeNode object at 0x1091294e0>]}
newFreqSet: {'y'}
bigL: ['z', 'x', 't']
myHead: None
newFreqSet: {'z', 'y'}
myHead: {'z': [3, <__main__.treeNode object at 0x109129550>]}
newFreqSet: {'y', 'x'}
bigL: ['z']
myHead: None
newFreqSet: {'z', 'y', 'x'}
conditional tree for: {'y', 'x'}
  Null Set  1
   z  3
myHead: {'z': [3, <__main__.treeNode object at 0x10912982

In [16]:
findPrefixPath('t',myHeaderTab['t'][1])

{frozenset({'x', 'z'}): 3}

## 12.5 示例：从新闻网站点击流中挖掘

In [27]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]
print(parsedDat[:6])
initSet = createInitSet(parsedDat)
# 寻找至少被10万人浏览过的新闻报道
myFPtree,myHeaderTab = createTree(initSet,100000)

[['1', '2', '3'], ['1'], ['4', '5', '6', '7'], ['1', '8'], ['9', '10'], ['11', '6', '12', '13', '14', '15', '16']]


In [28]:
myFreqList = []
mineTree(myFPtree,myHeaderTab,100000,set([]),myFreqList)

bigL: ['1', '3', '11', '6']
myHead: {'6': [107404, <__main__.treeNode object at 0x10dfadcf8>]}
newFreqSet: {'1'}
bigL: ['6']
myHead: None
newFreqSet: {'6', '1'}
conditional tree for: {'1'}
  Null Set  1
   6  107404
myHead: {'11': [127119, <__main__.treeNode object at 0x10dfb7668>], '6': [186289, <__main__.treeNode object at 0x10dfb7630>]}
newFreqSet: {'3'}
bigL: ['11', '6']
myHead: {'6': [117401, <__main__.treeNode object at 0x10dfc7588>]}
newFreqSet: {'3', '11'}
bigL: ['6']
myHead: None
newFreqSet: {'3', '11', '6'}
conditional tree for: {'3', '11'}
  Null Set  1
   6  117401
myHead: None
newFreqSet: {'3', '6'}
conditional tree for: {'3'}
  Null Set  1
   6  186289
    11  117401
   11  9718
myHead: {'6': [261773, <__main__.treeNode object at 0x10dfdbf60>]}
newFreqSet: {'11'}
bigL: ['6']
myHead: None
newFreqSet: {'11', '6'}
conditional tree for: {'11'}
  Null Set  1
   6  261773
myHead: None
newFreqSet: {'6'}
